# チーム統計
指定された大会および、シーズンに関するチームの統計情報を返却する<br>
シーズンに関して、チームがプレーした全ゲームのデータの統計を返す<br>
->チームidも必要

In [1]:
import requests
import pandas as pd
import numpy as np
from time import sleep
from pandas import json_normalize

## 一年分のデータのみ取得する場合

In [19]:
#パラメータとして、チームIdが必要。
# 取得済みのfixtures_YYYY.csvにシーズン別の試合結果が格納さている。その記録のうち、チームidのみリストとして取得する
df_fixtures = pd.read_csv('../../api_data/fixtures/fixtures_2023.csv')
teams = np.sort(df_fixtures['teams.home.id'].unique())

In [14]:
res = []
url = "https://api-football-v1.p.rapidapi.com/v3/teams/statistics"

headers = {
	"X-RapidAPI-Key": "xxxx",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

for i, id in enumerate(teams):
    # seisonに対象シーズン、teamに対象チームのidを格納
    querystring = {"league":"98","season":2023,"team":id}
    response = requests.get(url, headers=headers, params=querystring)
    res.append(response.json())


In [16]:
for i, id in enumerate(teams):
    if i == 0:
        df_status = json_normalize(res[i]['response'])
    else :
        df_status = pd.concat([df_status, json_normalize(res[i]['response'])])
    #2,lineups情報がリスト化されているため、df化
    # 2-1, lineupsに辞書で格納されているデータを辞書ごと・チーム別にスケールアップ。(explode)
    # 2-2, indexがチームごとに割り振られているため、リセット(reset_index(drop=True))
    df_lineup = df_status.reset_index(drop=True).explode('lineups', ignore_index=False)\
                    .reset_index(drop=True)
    # 2-3, 辞書を分解したときに発生するデータを格納するカラムを生成
    df_lineup[['formation','played']] = '0'
    # 2-4, 辞書型のデータをインデックスごとに分解・代入
    df_lineup[['formation','played']] = df_lineup['lineups'].apply(pd.Series)
    # 2-5. 分解した辞書列は必要ないため削除
    df_lineup = df_lineup.drop(columns='lineups')
    # 2-6. csvとして、データの保存
    df_lineup.to_csv("../../api_data/statistics/team/team_statistics_2023.csv")

## 複数年のデータを一括取得する場合

In [2]:
#teams配列に、シーズンごとのj1在籍チームを保有する。シーズンはすべてで8つあるため0配列を8つ作成
teams = [0]*8
for i in range(2016, 2024):
    # 取得済みのfixtures_YYYY.csvにシーズン別の試合結果が格納されている。その記録のうち、チームidのみリストとして取得する
    file_name = f"fixtures_{i}.csv"
    df_fixtures = pd.read_csv('C:/python/soccer_statistics/api_data/fixtures/' + file_name)
    teams[i-2016] = np.sort(df_fixtures['teams.home.id'].unique())

In [3]:
# シーズン数 * チーム数の０行列を作成
res = [[0]*20]*8
url = "https://api-football-v1.p.rapidapi.com/v3/teams/statistics"

headers = {
	"X-RapidAPI-Key": "xxxx",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

for i in range(len(teams)):
    for j in range(len(teams[i])):
        # seisonに対象シーズン、teamに対象チームのidを格納
        querystring = {"league":"98","season":i+2016,"team":teams[i][j]}
        response = requests.get(url, headers=headers, params=querystring)
        res[i][j] = response.json()
    #api呼び出し回数が毎分30回までであるため、待機
    sleep(61)

In [4]:
for i in range(len(teams)):
    #1,シーズンごとのチーム詳細のデータフレーム化
    for j in range(len(teams[i])):
        if i == 0:
            if j == 0:
                df_status = json_normalize(res[i][j]['response'])
            else :
                df_status = \
                pd.concat([df_status, json_normalize(res[i][j]['response'])])
    #2,lineups情報がリスト化されているため、df化
    # 2-1, lineupsに辞書で格納されているデータを辞書ごと・チーム別にスケールアップ。(explode)
    # 2-2, indexがチームごとに割り振られているため、リセット(reset_index(drop=True))
    df_lineup = df_status.reset_index(drop=True).explode('lineups', ignore_index=False)\
                    .reset_index(drop=True)
    # 2-3, 辞書を分解したときに発生するデータを格納するカラムを生成
    df_lineup[['formation','played']] = '0'
    # 2-4, 辞書型のデータをインデックスごとに分解・代入
    df_lineup[['formation','played']] = df_lineup['lineups'].apply(pd.Series)
    # 2-5. 分解した辞書列は必要ないため削除
    df_lineup = df_lineup.drop(columns='lineups')
    # 2-6. csvとして、データの保存
    file_name = f"team_statistics_{i+2016}.csv"
    df_lineup.to_csv("../../api_data/statistics/team/" + file_name,index=False)